# HardCode transformations

In [1]:
from pyspark.sql.functions import col, concat, initcap, trim, lit, substring_index, split
from pyspark.sql import functions as F

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 3, Finished, Available, Finished)

## Dimension tables
### Actor

In [2]:
# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/actor')

# Initial capitalize and unifying first and last names
df = df.withColumn("name", initcap(trim(concat(col('first_name'),lit(" "),col('last_name')))))

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Drop unnecesary columns
df = df.drop("first_name", "last_name")

# Re-order columns
df = df['actor_id','name','last_update'] 

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('actor')

# Sample show
df.show(3)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 4, Finished, Available, Finished)

+--------+----------------+-----------+
|actor_id|            name|last_update|
+--------+----------------+-----------+
|       1|Penelope Guiness| 2006-02-15|
|       2|   Nick Wahlberg| 2006-02-15|
|       3|        Ed Chase| 2006-02-15|
+--------+----------------+-----------+
only showing top 3 rows



### Address, City and Country

In [3]:
# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/address')

# De-normalization, unify Adress with City and Country
df_city = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/city')
df_country = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/country')
df_city_country = df_city.join(df_country.select('country_id','country'),'country_id',how='inner')

df = df.join(df_city_country.select('city_id','city','country'),'city_id',how='inner')

# Drop unnecesary columns
df = df.drop("address2", "district", "phone", "city_id", "country_id")

# Convert the postal_code col to int
df = df.withColumn('postal_code', col('postal_code').cast('int'))

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Clean possible blank spaces in address
df = df.withColumn('address', trim(col('address')))
df = df.withColumn('city', trim(col('city')))
df = df.withColumn('country', trim(col('country')))

# Full address, for future reporting uses
df = df.withColumn('full_address', trim(concat(
    col('address'), 
    lit(' '), 
    col('postal_code'), 
    lit(', '), 
    initcap(col('city')), 
    lit(', '), 
    initcap(col('country')))))

# Re-order columns
df = df['address_id','address', 'postal_code', 'city', 'country', 'full_address', 'last_update'] 

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('address')

# Sample show
df.show(3)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 5, Finished, Available, Finished)

+----------+--------------------+-----------+------------------+--------------------+--------------------+-----------+
|address_id|             address|postal_code|              city|             country|        full_address|last_update|
+----------+--------------------+-----------+------------------+--------------------+--------------------+-----------+
|        56|939 Probolinggo Loop|       4166|A Corua (La Corua)|               Spain|939 Probolinggo L...| 2006-02-15|
|       105|733 Mandaluyong P...|      77459|              Abha|        Saudi Arabia|733 Mandaluyong P...| 2006-02-15|
|       457|535 Ahmadnagar Manor|      41136|         Abu Dhabi|United Arab Emirates|535 Ahmadnagar Ma...| 2006-02-15|
+----------+--------------------+-----------+------------------+--------------------+--------------------+-----------+
only showing top 3 rows



### Customer

In [4]:
# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/customer')

# Initial capitalize and unifying first and last names
df = df.withColumn("name", initcap(trim(concat(col('first_name'),lit(" "),col('last_name')))))

# Mask the email address
df = df.withColumn("email", concat(
    substring_index(col('email'),'@',1).substr(1,1),
    lit('******@'),
    substring_index(col('email'),'@',-1)

))

# Convert active col into a boolean type
df = df.withColumn("active", col('active').cast('boolean'))

# Turn the create_date into cadastral_date
df = df.withColumn("create_date", col('create_date').cast('date'))
df = df.withColumnRenamed("create_date", "cadastral_date")

# Drop unnecesary columns
df = df.drop("first_name", "last_name")

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Re-order columns
df = df['customer_id','name', 'email', 'address_id', 'active', 'store_id', 'cadastral_date', 'last_update'] 

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('customer')

# Sample show
df.show(3)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 6, Finished, Available, Finished)

+-----------+-------------+--------------------+----------+------+--------+--------------+-----------+
|customer_id|         name|               email|address_id|active|store_id|cadastral_date|last_update|
+-----------+-------------+--------------------+----------+------+--------+--------------+-----------+
|         16|Sandra Martin|S******@sakilacus...|        20| false|       2|    2006-02-14| 2006-02-15|
|         64|   Judith Cox|J******@sakilacus...|        68| false|       2|    2006-02-14| 2006-02-15|
|        124| Sheila Wells|S******@sakilacus...|       128| false|       1|    2006-02-14| 2006-02-15|
+-----------+-------------+--------------------+----------+------+--------+--------------+-----------+
only showing top 3 rows



### Film, Category, Film Category and Language

In [5]:
# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/film')

# De-normalization, unify category, film category and lenguage with Film
df_category = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/category')
df_film_category = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/film_category')
df_language = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/language')

df_category_film_category = df_film_category.join(df_category.select('category_id','name'), 'category_id', how='inner')
df = df.join(df_category_film_category.select('film_id', 'name'),'film_id', how='inner')
df = df.withColumnRenamed('name', 'category')
df = df.join(df_language.select('language_id', 'name'), 'language_id', how='inner')
df = df.withColumnRenamed('name', 'language')

# pivot of special_features

# Explode the special_features column to get one feature per row
df_special_features = df.select('film_id',split('special_features',',').alias('special_features'))
exploded_df = df_special_features.withColumn("special_feature", F.explode(F.col("special_features")))

# Get distinct special features
special_features = exploded_df.select("special_feature").distinct()
special_features = special_features.rdd.flatMap(lambda x: x).collect()
print('Unique special features -> ', special_features)

# Loop through each unique special feature and create a new column with 1 or 0
for feature in special_features:
    df_special_features = df_special_features.withColumn(feature, F.when(F.array_contains(F.col("special_features"), feature), 1).otherwise(0))

# Merge the pivot
df = df.join(df_special_features, 'film_id', how='inner')

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Drop unnecesary columns
df = df.drop("language_id", "original_language_id", "special_features")

# Rename columns
df = df.withColumnsRenamed({
    'rental_duration':'rental_duration_days',
    'length':'length_mins',
    'Deleted Scenes':'deleted_scenes',
    'Behind the Scenes':'behind_the_scenes',
    'Commentaries':'commentaries',
    'Trailers':'trailers'
})

# Re-order columns
df = df['film_id', 'title', 'description', 'category', 'language', 'release_year', 'rental_duration_days', 'rental_rate', 'length_mins', 'replacement_cost', 
    'rating', 'deleted_scenes', 'behind_the_scenes', 'commentaries', 'trailers', 'last_update']

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('film')

# Sample show
display(df)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 7, Finished, Available, Finished)

Unique special features ->  ['Deleted Scenes', 'Behind the Scenes', 'Commentaries', 'Trailers']


SynapseWidget(Synapse.DataFrame, 0ce583d6-4a07-4d1a-be8e-6e9592201c6b)

### Film actor

In [6]:
# This tables are relational ones, many to many

# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/film_actor')

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('film_actor')

# Sample show
df.show(3)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 8, Finished, Available, Finished)

+--------+-------+-----------+
|actor_id|film_id|last_update|
+--------+-------+-----------+
|       1|      1| 2006-02-15|
|       1|     23| 2006-02-15|
|       1|     25| 2006-02-15|
+--------+-------+-----------+
only showing top 3 rows



### Inventory

In [7]:
# This tables are relational ones, many to many

# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/inventory')

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('inventory')

# Sample show
df.show(3)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 9, Finished, Available, Finished)

+------------+-------+--------+-----------+
|inventory_id|film_id|store_id|last_update|
+------------+-------+--------+-----------+
|           1|      1|       1| 2006-02-15|
|           2|      1|       1| 2006-02-15|
|           3|      1|       1| 2006-02-15|
+------------+-------+--------+-----------+
only showing top 3 rows



### Store

In [8]:
# This tables are relational ones, many to many

# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/store')

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('store')

# Sample show
df.show(3)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 10, Finished, Available, Finished)

+--------+----------------+----------+-----------+
|store_id|manager_staff_id|address_id|last_update|
+--------+----------------+----------+-----------+
|       1|               1|         1| 2006-02-15|
|       2|               2|         2| 2006-02-15|
+--------+----------------+----------+-----------+



### Staff

In [9]:
# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/staff')

# Initial capitalize and unifying first and last names
df = df.withColumn("name", initcap(trim(concat(col('first_name'),lit(" "),col('last_name')))))

# Mask the email address
df = df.withColumn("email", concat(
    substring_index(col('email'),'@',1).substr(1,1),
    lit('******@'),
    substring_index(col('email'),'@',-1)

))

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Drop unnecesary columns
df = df.drop("first_name", "last_name", "picture", "password")

# Re-order columns
df = df['staff_id', 'name', 'address_id', 'email', 'store_id', 'active', 'username', 'last_update']

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('staff')

# Sample show
display(df)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a871bdd8-f4e4-4176-abc3-81191fedf3f4)

## Fact Tables
### Payment

In [11]:
# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/payment')

# Assuming your date column is called 'date_column'
df = df.withColumn("payment_year", F.year(F.col("payment_date"))) \
       .withColumn("payment_quarter", F.quarter(F.col("payment_date"))) \
       .withColumn("payment_month", F.month(F.col("payment_date"))) \
       .withColumn("payment_day", F.dayofmonth(F.col("payment_date"))) \
       .withColumn("payment_hour", F.hour(F.col("payment_date")))

# Convert the last_update col to date
df = df.withColumn('payment_date', col('payment_date').cast('date'))
    
# Rename payment date column
df = df.withColumnRenamed('payment_date', 'payment_datetime')

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Re-order columns
df = df['payment_id', 'customer_id', 'staff_id', 'rental_id', 'amount', 'payment_datetime', 'payment_year', 'payment_quarter', 'payment_month',
    'payment_day', 'payment_hour', 'last_update']

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('payment')

# Sample show
display(df)

StatementMeta(, 25e096bd-9bde-4081-a791-2ab1b9b78cd3, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7dc7dea6-e990-4867-8dfe-80f723467f2b)

### Rental

In [ ]:
# Load df
df = spark.read.format('delta').load('abfss://sakila_project_dev@onelake.dfs.fabric.microsoft.com/DLH_bronze.Lakehouse/Tables/rental')

# Assuming your date column is called 'date_column'
df = df.withColumn("rental_year", F.year(F.col("rental_date"))) \
       .withColumn("rental_quarter", F.quarter(F.col("rental_date"))) \
       .withColumn("rental_month", F.month(F.col("rental_date"))) \
       .withColumn("rental_day", F.dayofmonth(F.col("rental_date"))) \
       .withColumn("rental_hour", F.hour(F.col("rental_date")))

# Rename rental date column
df = df.withColumnsRenamed({'rental_date':'rental_datetime', 'return_date':'return_datetime'})

# Convert the last_update col to date
df = df.withColumn('last_update', col('last_update').cast('date'))

# Re-order columns
df = df['rental_id', 'inventory_id', 'staff_id', 'customer_id', 'rental_datetime', 'rental_year', 'rental_quarter', 'rental_month', 'rental_day', 
    'rental_hour', 'return_datetime', 'last_update']

# Load as a delta table in the silver lakehouse
df.write.format("delta").mode("overwrite").saveAsTable('rental')

# Sample show
display(df)

StatementMeta(, , , Cancelled, , Cancelled)